# Работа со строковыми значениями

Материалы:
* Макрушин С.В. Лекция 8: Работа со строковыми значениям
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
* https://tproger.ru/translations/regular-expression-python/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {  
         'home_page': 'https://github.com/pypa/sampleproject',
         'keywords': 'sample setuptools development',
         'license': 'MIT'
      }

2. Дана строка 'aaa--bbb==ccc__ddd'. Написать регулярное выражение для разбивки строки на список ['aaa','bbb','ccc','ddd'].

3. Проверить корректность введенного E-mail

4. Разбейте текст формулировки задачи 1 на слова.

## Лабораторная работа 8

### Форматирование строк

In [2]:
from bs4 import BeautifulSoup
import re
import pandas as pd

1.1 Загрузите данные из файла `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в таблицу `recipes` При помощи форматирования строк выведите информацию об id рецепта и кол-ве игредиентов 5 случайных рецептов в виде таблицы следующего вида:

    
    |    id     | n_in  |
    |-------------------|
    |  400894   |  13   |
    |   68588   |   8   |
    |  362081   |   6   |
    |   53408   |  12   |
    |  221203   |   4   |

In [3]:
print(f'[Task 1.1] Answer:\n\n')

recipes = pd.read_csv('./data/recipes_sample_with_filled_tags_ingredients.csv', index_col = 0) # data
randomId = recipes.sample(n = 5, random_state = 2) # 5 random id 

longestIdLen = max([len(str(id_)) for id_ in randomId.index]) + 5 # the longest id
longestN_ingLen = max([len(str(n_ing)) for n_ing in randomId['n_ingredients']]) + 5 # the longest n_ing

print(f"|{'id': ^{longestIdLen}}|{'n_in': ^{longestN_ingLen}}|\n{'-'*(longestIdLen + longestN_ingLen + 3)}") # heading and line

for index, row in randomId.iterrows():
    print(f"|{index: ^{longestIdLen}}|{row['n_ingredients']: ^{longestN_ingLen}}|") # formatting

[Task 1.1] Answer:


|    id    | n_in  |
--------------------
|   7945   |   8   |
|  16536   |   8   |
|  26726   |   9   |
|  16333   |  16   |
|  20629   |   8   |


1.2 Напишите функцию `show_info`, которая для рецепта по его `id` создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название"

1. Шаг 1.
2. Шаг 2.
----------
#тэг1 #тэг2
```

    
Данные для создания строки получите из файлов `recipes_sample_with_tags_ingredients.csv`, `steps_sample.xml` (__ЛР4__) и `tags_sample.csv` (__ЛР5__). 
Выведите созданную строку на экран.

In [4]:
print(f'[Task 1.2] Answer:\n')

with open('./data/steps_sample.xml') as file:
    st = BeautifulSoup(file, 'xml')
    
steps = {}
for j in st.recipes.find_all("recipe"):
    stps = [step.next for step in j.steps.find_all("step")]
    steps[int(j.find("id").next)] = stps

[Task 1.2] Answer:



In [5]:
def show_info(rec_id):
    recipe = '"' + recipes[recipes['id'] == rec_id]['name'].values[0] + '"'
    ansSteps = '\n'.join([' '.join([f'{ind+1}.', step.capitalize()]) + '.' for ind, step in enumerate(steps[rec_id])])
    longestStepLen = max([len(step) for step in steps[rec_id]])
    lenN_steps = len(str(len(steps[rec_id])))
    line = '-' * (2 + lenN_steps + longestStepLen)
    tags = ' '.join(['#' + tag for tag in recipes[recipes['id'] == rec_id]['tags'].values[0].split('; ')])
    return recipe + '\n' + ansSteps + '\n' + line + '\n' + tags

print(show_info(44123)) # 44123 - id

"george s at the cove  black bean soup"
1. In 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes.
2. Add thyme , oregano and basil.
3. Saute 5 minutes more.
4. Add wine and deglaze pan.
5. Add hot chicken stock and reduce by one-third.
6. Add worcestershire sauce , tabasco , smoked chicken , beans and broccoli florets.
7. Simmer 5 minutes.
8. Add cream , simmer 5 minutes more and season to taste.
9. Drop in remaining butter , piece by piece , stirring until melted and serve immediately.
10. Smoked chicken: on a covered grill , slightly smoke boneless chicken , cooking to medium rare.
11. Chef meskan uses applewood chips and does not allow the grill to become too hot.
-------------------------------------------------------------------------------------------------
#weeknight #time-to-make #course #main-ingredient #cuisine #preparation #occasion #north-american #soups-stews #beans #poultry #american #chicken #stove-top #dietary #gluten-free #comfort-food #

## Работа с регулярными выражениями

В задачах данного блока подразумевается, что вы не будете использовать никаких строковые методы (`split`, `startswith` и т.д.). Все задачи необходимо решить при помощи регулярных выражений.

2.1 Посчитайте кол-во отзывов, в которых встречаются числа.

In [6]:
print(f'[Task 2.1] Answer:\n')

reviews = pd.read_csv('./data/reviews_sample.csv', index_col = 0)
reviews['review'] = reviews['review'].apply(str) # to string 
print(sum([1 if re.search(r'\d+', review) else 0 for review in reviews['review']]))

[Task 2.1] Answer:

49246


2.2 Найдите все смайлики в отзывах. Смайлик состоит из трех частей: глаза (символ `=` или `:`), нос (символ `-`), губы (символ `)` или `(`). Смайлик может иметь вид "глаза-нос-губы" или "губы-нос-глаза". Нос может отсутствовать.

In [7]:
print(f'[Task 2.2] Answer:\n')

# print(set(re.findall(r'(?:[=:]-?[()])|(?:[()]-?[:=])', ''.join(reviews['review']))))

sm = []
for row in reviews['review']:
    r = re.findall(re.compile("[=:]-{0,1}[)(]|[)(]-{0,1}[=:]"), str(row))
    sm.extend(r)

print(set(sm))

[Task 2.2] Answer:

{'=-)', '(=', '):', '(:', ':)', ':-(', ':(', '=)', ':-)', '=(', '(-:'}


2.3 Проверьте, что все даты в датасете имеют вид "YYYY-MM-DD". Продемонстрируйте работу вашего решения, приведя пример из датасета и контрпример не из датасета.

In [8]:
print(f'[Task 2.3] Answer:\n')

print(all([re.fullmatch(r'\d{4}-\d{2}-\d{2}', date) for date in reviews['date']]))

[Task 2.3] Answer:

True


2.4 Используя строку-результат задачи 1.2, найдите первое слово каждого шага в рецепте

In [9]:
print(f'[Task 2.4] Answer:\n')

a = re.findall(r'(?<=\d\. )[a-zA-Z]+', show_info(44123))
for i in a:
    print(i, end = ' ')


[Task 2.4] Answer:

In Add Saute Add Add Add Simmer Add Drop Smoked Chef 

2.5 Используя регулярные выражения, удалите из описаний все символы, кроме английских букв, цифр и пробелов. Сохраните предобработанные описания в файл `preprocessed_descriptions.csv`, содержащий 2 столбца: `name` и `preprocessed_descriptions`.

In [10]:
print(f'[Task 2.5] Answer:\n\n')

preprocessed_descriptions = []

for index, row in recipes.iterrows():
    preprocessed_descriptions.append(re.sub(r"[^a-zA-Z0-9 ]+", '', str(row['description'])))
recipes["preprocessed_descriptions"] = preprocessed_descriptions

recipes[['name','preprocessed_descriptions']].to_csv('./out/preprocessed_descriptions.csv')

[Task 2.5] Answer:




### Сегментация текста

In [11]:
import nltk
import string

3.1 Разбейте отзывы из файла `recipes_sample.csv` (__ЛР2__) на предложения, а предложения - на слова (используйте `sent_tokenize` и `word_tokenize` из `nltk`). Каждый отзыв представьте в виде списка списков: внешний список - предложения, вложенные списки - слова в предложении. Исключите знаки препинания из списков слов.

`'Предложение номер один. Предложение номер два.' => [['Предложение', 'номер', 'один'], ['Предложение', 'номер', 'два']]`

In [12]:
print(f'[Task 3.1] Answer:\n\n')

sentenceS = []
punctuations = list(string.punctuation)
for description in recipes['description']:
    descSentenceS = []
    for sentence in nltk.sent_tokenize(str(description)):
        sentence = nltk.word_tokenize(sentence)
        sentence = [word for word in sentence if word not in punctuations]
        descSentenceS.append(sentence)
    sentenceS.append(descSentenceS)
        
recipes['sentences'] = sentenceS

sentenceS

[Task 3.1] Answer:




[[['an',
   'original',
   'recipe',
   'created',
   'by',
   'chef',
   'scott',
   'meskan',
   'george',
   "'s",
   'at',
   'the',
   'cove'],
  ['we',
   'enjoyed',
   'this',
   'when',
   'we',
   'visited',
   'this',
   'restaurant',
   'in',
   'la',
   'jolla',
   'california'],
  ['this',
   'recipe',
   'is',
   'requested',
   'so',
   'often',
   'they',
   'have',
   'it',
   'printed',
   'and',
   'ready',
   'at',
   'the',
   'hostess',
   'stand'],
  ['it',
   "'s",
   'unbeatable',
   'at',
   'the',
   'restaurant',
   'but',
   'i',
   'do',
   'a',
   'pretty',
   'good',
   'job',
   'at',
   'home',
   'too',
   'if',
   'i',
   'do',
   'say',
   'so',
   'myself']],
 [['my',
   'children',
   'and',
   'their',
   'friends',
   'ask',
   'for',
   'my',
   'homemade',
   'popsicles',
   'morning',
   'noon',
   'and',
   'night'],
  ['i',
   'never',
   'turn',
   'them',
   'down',
   'who',
   'am',
   'i',
   'to',
   'tell',
   'them',
   'that',
   '

3.2 Посчитайте кол-во уникальных слов среди всех отзывов (без учета регистра и знаков препинания).

In [13]:
print(f'[Task 3.2] Answer:\n')

unique = set()
for description in sentenceS:
    for sentence in description:
        for word in sentence:
            unique.add(word.lower())
print(len(unique))

[Task 3.2] Answer:

29712


3.3 Найдите 5 самых длинных (по количеству слов; без учета знаков препинания) отзывов в датасете и выведите их в порядке убывания длины.

In [14]:
print(f'[Task 3.3] Answer:\n')

for index, row in recipes.iterrows():
    setOfuniqueWords = set()
    for sentence in row['sentences']:
        for word in sentence:
            setOfuniqueWords.add(word)
    recipes.loc[index, 'desc_word_count'] = len(setOfuniqueWords)
    
recipes.sort_values(by = 'desc_word_count', ascending = False)[:5]['description'] # off ascending

[Task 3.3] Answer:



18408    this wonderful icing is used for icing cakes a...
481      a translucent golden-brown crust allows the gr...
14136    this is the recipe as taught in the main line ...
401      ajiaco is a cuban stew consisting of different...
14935    the sweet of all sweet..............to quench ...
Name: description, dtype: object

3.4 Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на любом предложении из отзывов.


In [15]:
print(f'[Task 3.4] Answer:\n\n')

[Task 3.4] Answer:




#### [версия 2]
* Уточнены формулировки задач 3.1-3.3